# KDD Cup MOOC Dataset

- dataset preparation and exploration
- dataset preparation step is similar to XuetangX dataset
- this dataset doesn't provide user demographics (age, sex, education, etc.)

In [74]:
import os
import pandas as pd
import numpy as np
import pickle as pkl
import math
from sklearn.preprocessing import StandardScaler

In [75]:
#Modified version of original researcher's .sh file. Edited and implemented with the assistance of ChatGPT-4o

import os
import urllib.request
import tarfile
import zipfile
from tkinter import Tk, filedialog

#Hide the main tkinter window
root = Tk()
root.withdraw()

#Ask the user to select the destination folder
download_folder = filedialog.askdirectory(title="Select Destination Folder to Download Data")

#If no folder is selected, exit the script
if not download_folder:
    print("No folder selected. Exiting...")
    exit()

#Define file URLs and paths relative to the selected folder
files_info = {
    f"{download_folder}/kddcup15.zip": "http://lfs.aminer.cn/misc/moocdata/data/kddcup15.zip",
    f"{download_folder}/kdd2_test.csv": "https://bitbucket.org/lics229/mooc-dropout-prediction/raw/8742cb34f2453955c474aa0a50df72d1d59b39f5/data/test/FeatureVectorWithLabel.csv",
    f"{download_folder}/kdd2_train.csv": "https://bitbucket.org/lics229/mooc-dropout-prediction/raw/8742cb34f2453955c474aa0a50df72d1d59b39f5/data/train/FeatureVectorWithLabel.csv",
    f"{download_folder}/kdd2_valid.csv": "https://bitbucket.org/lics229/mooc-dropout-prediction/raw/8742cb34f2453955c474aa0a50df72d1d59b39f5/data/validation/FeatureVectorWithLabel.csv"
}

#Create the directory if it doesn't exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

#Function to download a file
def download_file(url, file_path):
    print(f"Downloading {file_path}...")
    urllib.request.urlretrieve(url, file_path)
    print(f"Downloaded {file_path} successfully.")

#Function to extract a tar.gz file
def extract_tar(file_path, extract_to=download_folder):
    print(f"Extracting files from {file_path}...")
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=extract_to)
    print(f"Done extracting files from {file_path}.")

#Function to extract a zip file
def extract_zip(file_path, extract_to= download_folder):
    print(f"Extracting files from {file_path}...")
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Done extracting files from {file_path}.")

#Loop through files and check if they exist, download and extract if necessary
for file_path, url in files_info.items():
    if os.path.exists(file_path):
        print(f"{file_path} exists.")
    else:
        download_file(url, file_path)
        
        # Check for compressed file types and extract
        if file_path.endswith(".tar.gz"):
            extract_tar(file_path)
        elif file_path.endswith(".zip"):
            extract_zip(file_path)

print("All done...")


C:/Users/chanc/Downloads/Module 5 Data/kddcup15.zip exists.
Downloaded C:/Users/chanc/Downloads/Module 5 Data/kdd2_test.csv successfully.
Downloaded C:/Users/chanc/Downloads/Module 5 Data/kdd2_train.csv successfully.
Downloaded C:/Users/chanc/Downloads/Module 5 Data/kdd2_valid.csv successfully.
All done...


In [76]:
#Unzip all folders within the kddcup15 folder
active_folder = download_folder + "/kddcup15"
for file in os.listdir(active_folder):
    if file.endswith(".zip"):
        with zipfile.ZipFile(active_folder + "/" + file, 'r') as zip_ref:
            zip_ref.extractall(active_folder)



In [77]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 20

In [78]:
# load training log
train_df = pd.read_csv(os.path.join(active_folder, 'train/log_train.csv'))

In [79]:
train_df.head()

,enrollment_id,time,source,event,object
0,1,2014-06-14T09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
1,1,2014-06-14T09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
2,1,2014-06-14T09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4
3,1,2014-06-14T09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W
4,1,2014-06-14T09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf


In [80]:
train_df.tail()

,enrollment_id,time,source,event,object
8157272,200901,2014-07-24T14:11:31,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
8157273,200901,2014-07-24T14:11:32,browser,video,HdMvr3A6vQzym6Xl0tOXpNbfHOyohlKE
8157274,200901,2014-07-24T14:11:41,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
8157275,200904,2014-07-24T15:07:50,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
8157276,200905,2014-07-24T15:41:51,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG


In [81]:
# read the ground truch for training data
train_truth_df = pd.read_csv(os.path.join(active_folder, 'train/truth_train.csv'), header=None, index_col=0)

In [82]:
train_truth_df.head()

,1
0,
1,0
3,0
4,0
5,0
6,0


In [83]:
train_truth_df.columns = ['truth']

In [84]:
train_truth_df.index.name = 'enrollment_id'

In [85]:
train_truth_df.tail()

,truth
enrollment_id,
200898,1
200900,1
200901,1
200904,1
200905,1


In [86]:
# load test logs
test_df = pd.read_csv(os.path.join(active_folder, 'test/log_test.csv'))
test_truth_df = pd.read_csv(os.path.join(active_folder, 'test/truth_test.csv'), header=None, index_col=0)

In [87]:
test_truth_df.columns = ['truth']
test_truth_df.index.name = 'enrollment_id'

# cobmine train and test truth
all_truth_df = pd.concat([train_truth_df, test_truth_df])

# combine train and test logs
all_log_df = pd.concat([train_df, test_df])

In [88]:
all_log_df.head()

,enrollment_id,time,source,event,object
0,1,2014-06-14T09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
1,1,2014-06-14T09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
2,1,2014-06-14T09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4
3,1,2014-06-14T09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W
4,1,2014-06-14T09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf


In [89]:
all_log_df.tail(10)

,enrollment_id,time,source,event,object
5387837,200894,2014-07-24T10:49:10,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
5387838,200894,2014-07-24T10:49:10,server,access,jcmKbpHQYLyzZZ34cJgLu7F4pgiCsoXV
5387839,200894,2014-07-24T10:54:17,browser,page_close,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl
5387840,200894,2014-07-24T10:54:17,browser,video,HdMvr3A6vQzym6Xl0tOXpNbfHOyohlKE
5387841,200894,2014-07-24T10:57:03,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387842,200899,2014-07-24T13:37:53,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387843,200902,2014-07-24T14:22:33,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387844,200903,2014-07-24T14:24:29,server,navigate,9Mur5ciTV9IBFfcPaz5c3nC1lrZaxBvG
5387845,200903,2014-07-24T14:24:41,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b
5387846,200903,2014-07-24T14:25:26,server,navigate,LMYZjRiU5C2N9ih1oYVNmOe5jFu2XLwv


In [90]:
# remove duplicate enroll_ids
train_enroll_ids = list(set(list(train_df['enrollment_id'])))
test_enroll_ids = list(set(list(test_df['enrollment_id'])))

In [91]:
# let's check total # of records on train and test datasets
print(len(train_enroll_ids))
print(len(test_enroll_ids))

120542
80362


In [92]:
# count all the actions for each user
user_action_count_df = all_log_df.groupby('enrollment_id').count()[['event']]

In [93]:
user_action_count_df.head(10)

,event
enrollment_id,
1,314
2,875
3,288
4,99
5,633
6,23
7,479
8,353
9,97


In [94]:
# give columns names
user_action_count_df.columns = ['action_count']

In [95]:
user_action_count_df.head(10)

,action_count
enrollment_id,
1,314
2,875
3,288
4,99
5,633
6,23
7,479
8,353
9,97


In [96]:
user_events_df = all_log_df[['event']].drop_duplicates()

In [97]:
user_events_df

,event
0,navigate
1,access
4,problem
12,page_close
69,video
484,discussion
2655,wiki


In [98]:
user_events_list = user_events_df['event'].values.tolist()

In [99]:
user_events_list

['navigate', 'access', 'problem', 'page_close', 'video', 'discussion', 'wiki']

In [100]:
sources_df = all_log_df[['source']].drop_duplicates()

In [101]:
sources_df

,source
0,server
4,browser


In [102]:
sources_list = sources_df['source'].values.tolist()

In [103]:
sources_list

['server', 'browser']

In [104]:
# Create composite action keys for efficient counting
all_log_df['source_event'] = all_log_df['source'] + '_' + all_log_df['event']

# Get unique combinations that we're interested in
source_event_combinations = [f'{source}_{action}' for source in sources_list for action in user_events_list]

# Create a crosstab (pivot table) of enrollment_id and source_event combinations
action_counts = pd.crosstab(
    index=all_log_df['enrollment_id'],
    columns=all_log_df['source_event'],
    values=1,
    aggfunc='sum'
).fillna(0)

# Rename columns to add "_count" suffix
action_counts.columns = [f"{col}_count" for col in action_counts.columns]

# Create a DataFrame with all required columns, using zeros for missing ones
# Get unique enrollment IDs
all_enrollment_ids = action_counts.index.unique()

# Create an empty DataFrame with the right index
complete_action_counts = pd.DataFrame(index=all_enrollment_ids)

# Add all required columns with zeros as default
for combo in source_event_combinations:
    column_name = f"{combo}_count"
    if column_name in action_counts.columns:
        complete_action_counts[column_name] = action_counts[column_name]
    else:
        complete_action_counts[column_name] = 0

# Merge with user_action_count_df in one operation
user_action_count_df = pd.merge(user_action_count_df, complete_action_counts, 
                               left_index=True, right_index=True, how='left')

In [105]:
# for source in sources_list:
#     for action in user_events_list:
#         action_label = f'{source}_{action}_count'
#         action_ = ((all_log_df['source'] == source) & (all_log_df['event'] == action)).astype(int)
#         #print(action_label)
#         #print(action_)
#         all_log_df[action_label] = action_
#         action_count = all_log_df.groupby(['enrollment_id']).sum()[[action_label]]
#         user_action_count_df = pd.merge(user_action_count_df, action_count, left_index=True, right_index=True)

In [106]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count
enrollment_id,,,,,,,,,,,,,,,
1,314,25.0,86.0,8.0,0,0,0.0,0.0,0,21.0,79.0,66.0,29.0,0,0
2,875,87.0,223.0,4.0,0,0,324.0,12.0,0,16.0,16.0,143.0,50.0,0,0
3,288,14.0,45.0,3.0,0,0,26.0,0.0,0,34.0,135.0,22.0,9.0,0,0
4,99,15.0,64.0,1.0,0,0,0.0,0.0,0,0.0,5.0,10.0,4.0,0,0
5,633,30.0,106.0,32.0,0,0,34.0,0.0,0,120.0,138.0,87.0,86.0,0,0
6,23,5.0,12.0,0.0,0,0,0.0,0.0,0,0.0,2.0,2.0,2.0,0,0
7,479,20.0,81.0,20.0,0,0,33.0,0.0,0,122.0,74.0,60.0,69.0,0,0
8,353,20.0,108.0,7.0,0,0,7.0,1.0,0,19.0,43.0,90.0,58.0,0,0
9,97,12.0,65.0,3.0,0,0,0.0,0.0,0,6.0,3.0,6.0,2.0,0,0


In [107]:
user_action_count_df.describe()

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count
count,200904.000000,200904.000000,200904.000000,200904.000000,200904.0,200904.0,200904.000000,200904.000000,200904.0,200904.000000,200904.000000,200904.000000,200904.000000,200904.0,200904.0
mean,67.420878,8.359679,19.618539,1.384213,0.0,0.0,5.357395,0.762100,0.0,6.096335,9.018432,10.258701,6.565484,0.0,0.0
std,139.918512,12.904720,38.431194,5.138802,0.0,0.0,35.715282,4.400607,0.0,21.728713,29.694653,20.919278,14.581310,0.0,0.0
min,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,5.000000,2.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,17.000000,4.000000,5.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,1.000000,0.0,0.0
75%,67.000000,10.000000,21.000000,1.000000,0.0,0.0,2.000000,1.000000,0.0,3.000000,5.000000,10.000000,6.000000,0.0,0.0
max,7697.000000,649.000000,3641.000000,483.000000,0.0,0.0,5321.000000,1041.000000,0.0,1122.000000,877.000000,694.000000,536.000000,0.0,0.0


In [108]:
user_action_count_df = pd.merge(user_action_count_df, all_truth_df, left_index=True, right_index=True)

In [109]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count,truth
enrollment_id,,,,,,,,,,,,,,,,
1,314,25.0,86.0,8.0,0,0,0.0,0.0,0,21.0,79.0,66.0,29.0,0,0,0
2,875,87.0,223.0,4.0,0,0,324.0,12.0,0,16.0,16.0,143.0,50.0,0,0,0
3,288,14.0,45.0,3.0,0,0,26.0,0.0,0,34.0,135.0,22.0,9.0,0,0,0
4,99,15.0,64.0,1.0,0,0,0.0,0.0,0,0.0,5.0,10.0,4.0,0,0,0
5,633,30.0,106.0,32.0,0,0,34.0,0.0,0,120.0,138.0,87.0,86.0,0,0,0
6,23,5.0,12.0,0.0,0,0,0.0,0.0,0,0.0,2.0,2.0,2.0,0,0,0
7,479,20.0,81.0,20.0,0,0,33.0,0.0,0,122.0,74.0,60.0,69.0,0,0,1
8,353,20.0,108.0,7.0,0,0,7.0,1.0,0,19.0,43.0,90.0,58.0,0,0,0
9,97,12.0,65.0,3.0,0,0,0.0,0.0,0,6.0,3.0,6.0,2.0,0,0,1


In [110]:
user_action_count_df.shape

(200904, 16)

In [111]:
numeric_features = [c for c in user_action_count_df.columns if 'count' in c or 'time' in c or 'num' in c]

In [112]:
numeric_features

['action_count',
 'server_navigate_count',
 'server_access_count',
 'server_problem_count',
 'server_page_close_count',
 'server_video_count',
 'server_discussion_count',
 'server_wiki_count',
 'browser_navigate_count',
 'browser_access_count',
 'browser_problem_count',
 'browser_page_close_count',
 'browser_video_count',
 'browser_discussion_count',
 'browser_wiki_count']

In [113]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(user_action_count_df[numeric_features])

In [114]:
for i, n_f in enumerate(numeric_features):
    print(i, n_f)
    user_action_count_df[n_f] = scaled_df[:,i]

0 action_count
1 server_navigate_count
2 server_access_count
3 server_problem_count
4 server_page_close_count
5 server_video_count
6 server_discussion_count
7 server_wiki_count
8 browser_navigate_count
9 browser_access_count
10 browser_problem_count
11 browser_page_close_count
12 browser_video_count
13 browser_discussion_count
14 browser_wiki_count


In [115]:
user_action_count_df.head(10)

,action_count,server_navigate_count,server_access_count,server_problem_count,server_page_close_count,server_video_count,server_discussion_count,server_wiki_count,browser_navigate_count,browser_access_count,browser_problem_count,browser_page_close_count,browser_video_count,browser_discussion_count,browser_wiki_count,truth
enrollment_id,,,,,,,,,,,,,,,,
1,1.762310,1.289479,1.727285,1.287421,0.0,0.0,-0.150003,-0.173181,0.0,0.685899,2.356712,2.664597,1.538584,0.0,0.0,0
2,5.771796,6.093934,5.292106,0.509028,0.0,0.0,8.921766,2.553722,0.0,0.455788,0.235113,6.345421,2.978788,0.0,0.0,0
3,1.576487,0.437075,0.660441,0.314429,0.0,0.0,0.577978,-0.173181,0.0,1.284187,4.242578,0.561268,0.166962,0.0,0.0,0
4,0.225697,0.514567,1.154832,-0.074767,0.0,0.0,-0.150003,-0.173181,0.0,-0.280567,-0.135325,-0.012367,-0.175944,0.0,0.0,0
5,4.042214,1.676935,2.247697,5.957782,0.0,0.0,0.801973,-0.173181,0.0,5.242094,4.343606,3.668458,5.447708,0.0,0.0,0
6,-0.317478,-0.260346,-0.198239,-0.269366,0.0,0.0,-0.150003,-0.173181,0.0,-0.280567,-0.236354,-0.394790,-0.313106,0.0,0.0,0
7,2.941570,0.902023,1.597182,3.622602,0.0,0.0,0.773973,-0.173181,0.0,5.334138,2.188331,2.377779,4.281829,0.0,0.0,1
8,2.041044,0.902023,2.299738,1.092823,0.0,0.0,0.045992,0.054061,0.0,0.593855,1.144369,3.811867,3.527437,0.0,0.0,0
9,0.211403,0.282093,1.180853,0.314429,0.0,0.0,-0.150003,-0.173181,0.0,-0.004434,-0.202678,-0.203578,-0.313106,0.0,0.0,1


In [116]:
user_action_count_df.loc[train_enroll_ids].to_csv(os.path.join(active_folder, 'kdd_train_normalized_features.csv'))
user_action_count_df.loc[test_enroll_ids].to_csv(os.path.join(active_folder, 'kdd_test_normalized_features.csv'))

In [117]:
# save single file with all features
user_action_count_df.to_csv(os.path.join(active_folder, 'kdd_all_normalized_features.csv'))

In [118]:
user_action_count_df.columns

Index(['action_count', 'server_navigate_count', 'server_access_count',
       'server_problem_count', 'server_page_close_count', 'server_video_count',
       'server_discussion_count', 'server_wiki_count',
       'browser_navigate_count', 'browser_access_count',
       'browser_problem_count', 'browser_page_close_count',
       'browser_video_count', 'browser_discussion_count', 'browser_wiki_count',
       'truth'],
      dtype='object')

## KDD Cup (Extended Features Dataset) Processing

In [120]:
#Merge the kdd2_train.csv and kdd2_test.csv files
train_df = pd.read_csv(os.path.join(download_folder, 'kdd2_train.csv'))
test_df = pd.read_csv(os.path.join(download_folder, 'kdd2_test.csv'))
valid_df = pd.read_csv(os.path.join(download_folder, 'kdd2_valid.csv'))
all_df = pd.concat([train_df, test_df, valid_df])
#Rename the label column to 'truth'
all_df.rename(columns={'label':'truth'}, inplace=True)
#sort all_df by enrollment_id
all_df.sort_values(by='enrollment_id', inplace=True)
#Reset the index of all_df
all_df.reset_index(drop=True, inplace=True)
#Save all_df to a CSV file in the selected folder
all_df.to_csv(os.path.join(download_folder, 'kdd_expanded_all.csv'))

In [121]:
train_df.shape

(72395, 141)

In [122]:
test_df.shape

(24013, 141)

In [128]:
valid_df.shape

(24134, 141)

In [123]:
all_df

,enrollment_id,truth,avg_chapter_delays,server_discussion_percent,act_cnt_weekDay_01,browser_html_percent,parallel_enrollments,browser_dictation,act_cnt_day_00,act_cnt_day_01,act_cnt_day_02,act_cnt_day_03,act_cnt_day_04,act_cnt_day_05,act_cnt_day_06,...,act_cnt_hour_21,act_cnt_hour_20,act_cnt_hour_23,server_access,server_outlink_percent,server_course_percent,browser_course_info_percent,browser_course,browser_vertical_percent,sessions_in_week_1,sessions_in_week_0,sessions_in_week_3,sessions_in_week_2,sessions_in_week_4,browser_about
0,1,0,0.161290,0,21,0,4,0,0,0,0,13,0,0,0,...,0,0,6,86,0,0,0,0,0,5,1,4,6,2,0
1,3,0,0.172414,0,10,0,0,0,0,28,0,0,0,3,0,...,0,0,10,45,0,0,0,0,0,2,2,1,1,1,0
2,4,0,0.250000,0,7,0,1,0,0,0,0,0,20,11,0,...,0,0,7,64,0,0,0,0,0,4,4,0,5,0,0
3,5,0,0.166667,0,77,0,0,0,0,2,0,145,0,0,0,...,34,23,63,106,0,0,0,0,0,16,2,13,1,4,0
4,6,0,0.035714,0,6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12,0,0,0,0,0,0,0,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120537,200898,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
120538,200900,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
120539,200901,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0,0,1,0,0,0
120540,200904,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [124]:
#Describe the dataset
all_df.describe()


,enrollment_id,truth,avg_chapter_delays,server_discussion_percent,act_cnt_weekDay_01,browser_html_percent,parallel_enrollments,browser_dictation,act_cnt_day_00,act_cnt_day_01,act_cnt_day_02,act_cnt_day_03,act_cnt_day_04,act_cnt_day_05,act_cnt_day_06,...,act_cnt_hour_21,act_cnt_hour_20,act_cnt_hour_23,server_access,server_outlink_percent,server_course_percent,browser_course_info_percent,browser_course,browser_vertical_percent,sessions_in_week_1,sessions_in_week_0,sessions_in_week_3,sessions_in_week_2,sessions_in_week_4,browser_about
count,120542.000000,120542.000000,120542.000000,120542.0,120542.000000,120542.0,120542.000000,120542.0,120542.0,120542.000000,120542.000000,120542.000000,120542.000000,120542.000000,120542.000000,...,120542.000000,120542.000000,120542.000000,120542.000000,120542.0,120542.0,120542.0,120542.0,120542.0,120542.000000,120542.000000,120542.000000,120542.000000,120542.000000,120542.0
mean,100364.330748,0.792927,0.084513,0.0,10.201025,0.0,0.408024,0.0,0.0,3.929303,3.463506,2.931028,2.395970,2.164565,2.122729,...,0.138110,0.132651,0.605805,19.669518,0.0,0.0,0.0,0.0,0.0,0.877022,1.058743,0.756483,0.829238,0.234814,0.0
std,58003.738782,0.405210,0.191068,0.0,33.999750,0.0,0.906857,0.0,0.0,17.630305,15.043837,13.156139,11.955223,11.334064,11.351597,...,2.570099,2.615489,5.496705,39.266200,0.0,0.0,0.0,0.0,0.0,1.985640,2.017998,1.924629,1.961478,0.824212,0.0
min,1.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,50060.250000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,100360.500000,1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,5.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,150658.500000,1.000000,0.088235,0.0,4.000000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,21.000000,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
max,200905.000000,1.000000,5.000000,0.0,3604.000000,0.0,11.000000,0.0,0.0,1478.000000,471.000000,360.000000,790.000000,430.000000,635.000000,...,238.000000,275.000000,527.000000,3641.000000,0.0,0.0,0.0,0.0,0.0,81.000000,86.000000,81.000000,71.000000,22.000000,0.0


In [125]:
#Drop non-numeric and unnecessary columns
non_features = ['enrollment_id']
numeric_features = all_df.drop(columns=non_features).select_dtypes(include=['number'])

#Identify features with binary values (0 or 1) to potentially exclude from scaling
binary_features = [col for col in numeric_features.columns if set(all_df[col].unique()).issubset({0, 1})]

#Identify continuous features for scaling
features_to_scale = [col for col in numeric_features.columns if col not in binary_features]
#Display first 10 features to be scaled for confirmation
features_to_scale[:10] 



['avg_chapter_delays',
 'act_cnt_weekDay_01',
 'parallel_enrollments',
 'act_cnt_day_01',
 'act_cnt_day_02',
 'act_cnt_day_03',
 'act_cnt_day_04',
 'act_cnt_day_05',
 'act_cnt_day_06',
 'act_cnt_day_07']

In [126]:
#Scale the continuous features
#Apply Standard Scaler to the selected continuous features
scaler = StandardScaler()
all_df_scaled = all_df.copy()

#Scale the continuous features
all_df_scaled[features_to_scale] = scaler.fit_transform(all_df[features_to_scale])




In [127]:
all_df_scaled

,enrollment_id,truth,avg_chapter_delays,server_discussion_percent,act_cnt_weekDay_01,browser_html_percent,parallel_enrollments,browser_dictation,act_cnt_day_00,act_cnt_day_01,act_cnt_day_02,act_cnt_day_03,act_cnt_day_04,act_cnt_day_05,act_cnt_day_06,...,act_cnt_hour_21,act_cnt_hour_20,act_cnt_hour_23,server_access,server_outlink_percent,server_course_percent,browser_course_info_percent,browser_course,browser_vertical_percent,sessions_in_week_1,sessions_in_week_0,sessions_in_week_3,sessions_in_week_2,sessions_in_week_4,browser_about
0,1,0,0.401837,0,0.317621,0,3.960922,0,0,-0.222873,-0.230229,0.765347,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,0.981355,1.689258,0,0,0,0,0,2.076406,-0.029110,1.685275,2.636167,2.141673,0
1,3,0,0.460054,0,-0.005913,0,-0.449933,0,0,1.365308,-0.230229,-0.222789,-0.200413,0.07371,-0.186999,...,-0.053737,-0.050718,1.709066,0.645099,0,0,0,0,0,0.565552,0.466433,0.126527,0.087058,0.928388,0
2,4,0,0.866122,0,-0.094149,0,0.652780,0,0,-0.222873,-0.230229,-0.222789,1.472503,0.77955,-0.186999,...,-0.053737,-0.050718,1.163282,1.128978,0,0,0,0,0,1.572788,1.457518,-0.393056,2.126345,-0.284897,0
3,5,0,0.429975,0,1.964698,0,-0.449933,0,0,-0.109431,-0.230229,10.798728,-0.200413,-0.19098,-0.186999,...,13.175379,8.743085,11.351246,2.198604,0,0,0,0,0,7.616205,0.466433,6.361519,0.087058,4.568243,0
4,6,0,-0.255399,0,-0.123561,0,-0.449933,0,0,-0.222873,-0.230229,-0.222789,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,-0.110213,-0.195322,0,0,0,0,0,-0.441684,-0.524652,0.126527,-0.422764,2.141673,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120537,200898,1,-0.442319,0,-0.300034,0,-0.449933,0,0,-0.222873,-0.230229,-0.222789,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,-0.110213,-0.500930,0,0,0,0,0,-0.441684,-0.524652,-0.393056,-0.422764,-0.284897,0
120538,200900,1,-0.442319,0,-0.300034,0,-0.449933,0,0,-0.222873,-0.230229,-0.222789,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,-0.110213,-0.500930,0,0,0,0,0,-0.441684,-0.524652,-0.393056,-0.422764,-0.284897,0
120539,200901,1,-0.442319,0,-0.300034,0,-0.449933,0,0,-0.222873,-0.230229,-0.222789,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,-0.110213,-0.424528,0,0,0,0,0,-0.441684,-0.524652,0.126527,-0.422764,-0.284897,0
120540,200904,1,-0.442319,0,-0.300034,0,-0.449933,0,0,-0.222873,-0.230229,-0.222789,-0.200413,-0.19098,-0.186999,...,-0.053737,-0.050718,-0.110213,-0.500930,0,0,0,0,0,-0.441684,-0.524652,-0.393056,-0.422764,-0.284897,0


In [129]:
# Save the scaled data to a new CSV file
all_df_scaled.to_csv(os.path.join(download_folder, 'kdd_expanded_all_scaled.csv'))


### Creating the final datasets

In [130]:
user_action_count_df.shape

(200904, 16)

In [131]:
all_df_scaled.shape

(120542, 141)

In [132]:
#Conduct an inner join on the two datasets to merge them
merged_df = pd.merge(left=all_df_scaled, right=user_action_count_df , left_on='enrollment_id', right_on='enrollment_id')

In [133]:
merged_df.shape

(120542, 157)

In [134]:
# Save the scaled data to a new CSV file
merged_df.to_csv(os.path.join(download_folder, 'kdd_merged_normalized_all.csv'))